In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import torch
from kiwipiepy import Kiwi
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

/home/jtkim1/anaconda3/envs/llmbrender/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize Kiwi for Korean sentence splitting
kiwi = Kiwi()

# Load the datasets
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
# Define a function to split queries into sentences using Kiwi
def split_into_sentences(query):
    return [item[0] for item in kiwi.split_into_sents(query)]

# Apply the function to the test dataset
test_df["split_queries"] = test_df["질문"].map(split_into_sentences)

# Template for generating prompts
PROMPT_TEMPLATE = """<s>[INST] <<SYS>>
You are an helpful assistant for construction and interior. Your task is to generate a valid answer based on the given information:

{context}

<</SYS>>
{query} [/INST]
"""

In [4]:
embedding_model_name = "osung-station/deco_embedding"
rerank_model_name = "Dongjin-kr/ko-reranker"
final_embedding_model_name = (
    "sentence-transformers/distiluse-base-multilingual-cased-v1"
)
final_embedding_model = SentenceTransformer(final_embedding_model_name)

# Set up the vector store for embeddings
chromadb_store = "vector_store/"
vectordb = Chroma(
    persist_directory=chromadb_store,
    embedding_function=HuggingFaceEmbeddings(model_name=embedding_model_name),
)

# # Load reranking model and tokenizer
rerank_tokenizer = AutoTokenizer.from_pretrained(rerank_model_name)
rerank_model = AutoModelForSequenceClassification.from_pretrained(rerank_model_name)
rerank_model.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [5]:
# Function to normalize scores
def exp_normalize(scores):
    b = scores.max()
    y = np.exp(scores - b)
    return y / y.sum()


# Function to get the best matching document
def get_first_place_doc(query, matching_docs):
    pairs = [[query, doc.page_content] for doc in matching_docs]

    with torch.no_grad():
        inputs = rerank_tokenizer(
            pairs, padding=True, truncation=True, return_tensors="pt", max_length=512
        )
        logits = rerank_model(**inputs, return_dict=True).logits.view(-1).float()
        scores = exp_normalize(logits.numpy())
        max_idx = scores.argmax()
        #print(scores[max_idx])

    return matching_docs[max_idx]

In [6]:
# Processing the queries from the test data
embeddings = []
generated_answers = []

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")
model = AutoModelForCausalLM.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:  60%|▌| 3/5 [00:21<00:14,  7.18s/it]

In [ ]:
def get_prompt(content, query):
    messages = [
        {
            "role": "system",
            "content": "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."
        },
        {
            "role": "user",
            "content": f"""information만 참고해서 question에 답해주세요.\n\n
            ```information : {first_place_doc.page_content.split("answer: ")[1]}\n\n question : {query}```"""
        }
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
from tqdm import tqdm

for queries in tqdm(test_df["split_queries"][:3]):
    generated_answer = []
    for query in tqdm(queries):
        # Get matching documents for the query
        matching_docs = vectordb.similarity_search(query, k=10)
        # Find the best document
        first_place_doc = get_first_place_doc(query, matching_docs)
        
        prompt = get_prompt(first_place_doc.page_content.split("answer: ")[1], query)
        input_text = tokenizer(prompt, return_tensors='pt').to("cuda")
            
        # Generate Text
        outputs = model.generate(
            **input_text, 
            max_new_tokens=256, 
            pad_token_id=32000, 
            num_beams=3, 
            top_p=0.92, 
            top_k =50, 
            do_sample=True, 
            temperature=0.1,
            early_stopping=True,
            repetition_penalty=1.5,
            no_repeat_ngram_size=2,
            encoder_repetition_penalty=0.8
        )
        text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        
        generated_answer.append(text)

        #generated_answer.append(first_place_doc.page_content.split("answer: ")[1])

    generated_answer = " ".join(generated_answer)
    embeddings.append(final_embedding_model.encode(generated_answer))
    generated_answers.append(generated_answer)

In [ ]:
test_df["generated_answer"] = generated_answers
test_df.to_csv("generated_test.csv", index=False)

ids = test_df["id"]
columns = ["id"] + [f"vec_{i}" for i in range(512)]
data = [[id] + list(embedding) for id, embedding in zip(ids, embeddings)]

submission = pd.DataFrame(data, columns=columns)
submission.to_csv("submission.csv", index=False)